Solution to homework 2: using UNET model for sematic segmentation 

Val_accuracy:  - test accuracy: 

parameters:

4 down sampling layers
4 up samling layers  
1 sigmiod layer  

optimizer: ADAM with lr = 0.0023
dropout: 50% 
Augmentation: no
 

Impoting data from googe drive and importing the useful libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [2]:
#try:
  # Use the %tensorflow_version magic if in colab.
  #%tensorflow_version 2.x
#except Exception:
  #pass

import tensorflow as tf
tf.enable_eager_execution()
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
import os
import cv2
import glob
import shutil
import numpy as np
import keras
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K
from keras.metrics import categorical_crossentropy
from keras.losses import binary_crossentropy
from tensorflow.python.keras import losses
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,SeparableConv2D, ZeroPadding2D,  Conv2DTranspose, BatchNormalization, Activation, Add, Conv2D, Flatten, LeakyReLU, MaxPooling2D, UpSampling2D, concatenate
from keras.applications.mobilenet import MobileNet
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_files  # to load the images from the subfolders    
from keras.utils import np_utils # to hot encode the label of the image
import matplotlib.pyplot as plt # to draw barplot and etc...
from tqdm import tqdm 
from keras.preprocessing import image    
from keras.callbacks import ModelCheckpoint # to save the best weights for the model while training 
import random as rn
from sklearn.utils import class_weight
from datetime import datetime
import cv2
from PIL import Image



Using TensorFlow backend.


Fixing a random seed to make the results reproducible

In [0]:
rand_seed = 1234
os.environ['PYTHONHASHSEED'] = '0'

#random seed for NP genreator of ranodm numbers
np.random.seed(rand_seed)

#random seed generator for Python
rn.seed(rand_seed)

#random seed for tensorflow
tf.set_random_seed(rand_seed)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

 since the data is not splitted into training and vailation sets. i will do so in the next code .The code below creates a train and a val folder each containing 20 folders (one for each type of class). It then moves the images from the original folders to these new folders such that 80% of the images go to the training set and 20% of the images go into the validation set. In the end our directory will have the 

Since I did nott delete the original folders, they will still be in the directory which is ok

In [0]:
train_list = []
val_list = []
train_dict = {}
val_dict = {}
############################################################### Generating JSON file for data Split ##########################################
#for image in glob.glob(os.path.join(base_dir, 'train', 'images') + '/*.jpg'):
    #  train_list.append(image)
   #   train_dict.setdefault (images , []).append(os.path.basename(image)) 
      #print(image)
      
#for image in glob.glob(os.path.join(base_dir, 'val', 'images') + '/*.jpg'):
 #     val_list.append(image)
  #    val_dict.setdefault (images , []).append(os.path.basename(image))   
   
#dataset_split = {'training' : train_dict , 'validation' : val_dict }

#print(dataset_split)
#import json
#with open('dataset_split.json', 'w') as fp:
#      json.dump(dataset_split, fp)

preparing the data

In [0]:
b_dir = '/content/drive/My Drive/Segmentation_Dataset_/training'

train_dir = os.path.join(b_dir, 'train')
val_dir = os.path.join(b_dir, 'val')

train_image_dir = os.path.join(train_dir, 'image')
train_mask_dir = os.path.join(train_dir, 'mask')

val_image_dir = os.path.join(val_dir, 'image')
val_mask_dir = os.path.join(val_dir, 'mask')


In [0]:
image_gen_train = ImageDataGenerator(rescale=1./255)
mask_gen_train = ImageDataGenerator(rescale=1./255)
image_gen_val = ImageDataGenerator(rescale=1./255)
mask_gen_val = ImageDataGenerator(rescale=1./255)

In [8]:
batch_size = 128
IMG_SHAPE = 256

train_img_gen = image_gen_train.flow_from_directory(
                                                directory= train_image_dir,
                                                shuffle=True,
                                                batch_size=8,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode=None,
                                                interpolation='bilinear',
                                                seed = rand_seed                                                
                                                )


train_mask_gen = mask_gen_train.flow_from_directory(
                                                batch_size=8,
                                                directory= train_mask_dir,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode=None,
                                                interpolation='bilinear',
                                                seed = rand_seed                                                
                                                )

train_gen = zip(train_img_gen, train_mask_gen)



val_img_gen = image_gen_val.flow_from_directory(
                                                 batch_size=4,
                                                 directory=val_image_dir,
                                                 shuffle=True,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode=None,
                                                 interpolation='bilinear',
                                                 seed = rand_seed
                                                 )

val_mask_gen = mask_gen_val.flow_from_directory(
                                                 batch_size=4,
                                                 directory=val_mask_dir,
                                                 shuffle=True,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode=None,
                                                 interpolation='bilinear',
                                                 seed = rand_seed
                                                 )

valid_gen = zip(val_img_gen, val_mask_gen)

Found 6116 images belonging to 1 classes.
Found 6116 images belonging to 1 classes.
Found 1531 images belonging to 1 classes.
Found 1531 images belonging to 1 classes.


In [9]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_SHAPE, IMG_SHAPE, 3], [None, IMG_SHAPE, IMG_SHAPE, 3]))
def prepare_target(x_, y_):
    y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), tf.int32)
    print (y_)
    return x_, y_

train_dataset = train_dataset.map(prepare_target)
print (train_dataset)

# Repeat
train_dataset = train_dataset.repeat()

Tensor("Cast:0", shape=(?, 256, 256, 1), dtype=int32)
<DatasetV1Adapter shapes: ((?, 256, 256, 3), (?, 256, 256, 1)), types: (tf.float32, tf.int32)>


In [10]:
# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_SHAPE, IMG_SHAPE, 3], [None, IMG_SHAPE, IMG_SHAPE, 3]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()
print (valid_dataset)


Tensor("Cast:0", shape=(?, 256, 256, 1), dtype=int32)
<DatasetV1Adapter shapes: ((?, 256, 256, 3), (?, 256, 256, 1)), types: (tf.float32, tf.int32)>


defining the Loss function 

In [0]:
def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax
    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

In [0]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_pred = tf.cast(y_pred > 0.5 , tf.float32) # when using sigmoid. Use argmax for softmax
    y_true = tf.cast(y_true, tf.float32) # when using sigmoid. Use argmax for softmax

    intersection = tf.reduce_sum(y_true * y_pred)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    return score

In [0]:
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

In [0]:
def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

building UNET model 

In [0]:
    inputs = tf.keras.layers.Input((IMG_SHAPE,IMG_SHAPE,3))
    conv1 = tf.keras.layers.Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',)(inputs)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
   
    conv4 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = tf.keras.layers.Dropout(0.5)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = tf.keras.layers.Dropout(0.5)(conv5)

    up6 = tf.keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(drop5))
    merge6 = tf.keras.layers.concatenate([drop4,up6], axis = 3)
    conv6 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = tf.keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv6))
    merge7 = tf.keras.layers.concatenate([conv3,up7], axis = 3)
    conv7 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = tf.keras.layers.Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv7))
    merge8 = tf.keras.layers.concatenate([conv2,up8], axis = 3)
    conv8 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = tf.keras.layers.Conv2D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv8))
    merge9 = tf.keras.layers.concatenate([conv1,up9], axis = 3)
    conv9 = tf.keras.layers.Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = tf.keras.layers.Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = tf.keras.layers.Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = tf.keras.layers.Conv2D(1, (1,1), activation = 'sigmoid')(conv9)

    model = tf.keras.Model(inputs = [inputs], outputs = [conv10])


In [0]:
model.summary()

In [0]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

Compiling and training the model 

In [0]:
epochs = 100
learning_rate = 0.0023
decay_rate = learning_rate / epochs

opt1 = tf.keras.optimizers.SGD(lr=learning_rate,
            momentum = .9, 
            decay = decay_rate,
            nesterov=True
           )

opt2 = adam(lr=learning_rate, 
            beta_1=0.9, beta_2=0.999, epsilon=None, 
            decay=decay_rate, amsgrad=False)

model.compile(optimizer = opt2,
              loss='binary_crossentropy',
              metrics=[dice_loss])

In [0]:

model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_loss])


In [0]:
 model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [my_IoU])

In [19]:
checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/weights.best.segmentation_acc.hdf5', 
                               monitor='val_loss',
                               verbose=1, 
                               save_best_only=True
                               )
                               
history = model.fit(train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=5,
          validation_data=valid_dataset,
          validation_steps=5,
          callbacks= [checkpointer])

Train for 5 steps, validate for 5 steps
Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
4/5 [=======================>......] - ETA: 2s - loss: 1.2017 - dice_loss: 0.5513
Epoch 00001: val_loss improved from inf to 1.22551, saving model to /content/drive/My Drive/weights.best.segmentation_acc.hdf5
5/5 [==============================] - 14s 3s/step - loss: 1.1799 - dice_loss: 0.5478 - val_loss: 1.2255 - val_dice_loss: 0.5262
Epoch 2/100
4/5 [=======================>......] - ETA: 0s - loss: 1.0774 - dice_loss: 0.5212
Epoch 00002: val_loss did not improve from 1.22551
5/5 [==============================] - 1s 250ms/step - loss: 1.0945 - dice_loss: 0.5398 - val_loss: 1.2400 - val_dice_loss: 0.5573
Epoch 3/100
4/5 [=======================>......] - ETA: 0s - loss: 1.1397 - dice_loss: 0.5684
Epoch 00003: val_loss did not improve from 1.22551
5/5 [==============================] - 1s 242ms/step - loss: 1.1003 - dice_loss: 0.5483 - val_l

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


KeyboardInterrupt: ignored

The remaining cells do proccesing to convert images to 4D, to predicat on the test data and to generate CSV submission for Kaggle. at the end the predicted image is plotted along with the respective model prediction  

In [0]:
model.load_weights('/content/drive/My Drive/weights.best.segmentation_acc.hdf5')

In [0]:
test = load_files("/content/drive/My Drive/Segmentation_Dataset_/test/images/img") #this line takes too much time on colab to execute 5 min 

In [0]:
def predict_img(img_path):
    img=path_to_tensor(img_path).astype('float32')/255
    # obtain predicted vector
    predicted_vector = model.predict(img)
    return predicted_vector

In [0]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(256, 256))
    # convert PIL.Image.Image type to 3D tensor with shape (256, 256, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 256, 256, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [0]:
def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

In [0]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
          f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

In [0]:
image_list = []
results = {}

for filename in glob.glob('/content/drive/My Drive/Segmentation_Dataset_/test/images/img/*.tif'): 
    image_list.append(filename)

for image_name in image_list:
    results[os.path.splitext(os.path.basename(image_name))[0]] = rle_encode(np.round((predict_img(image_name))))


create_csv(results)